<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E6%AF%8F%E4%B8%AA%E4%BA%BA%E7%89%A9%E6%80%BB%E7%BB%93profile_%E6%95%B0%E6%8D%AE%E5%87%86%E5%A4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 每个人获取前中后期的profile并抽取2000个token
- [ ] 连接task message 形成一次性的prompt

In [ ]:
def package_prefix_prompt( role_name, novel_name ):
    prefix_prompt = f"""请帮助我为小说{novel_name}中的 {role_name} 角色总结他的人物设定

给定人物{role_name}的对话数据，请为我总结{role_name}的人物设定、人物特点以及语言风格，并输出成json格式"""

    return prefix_prompt

def package_suffix_prompt( role_name , novel_name ):
    suffix_prompt = f"""现在让我们开始总结小说{novel_name} 中 {role_name} 的角色信息

Let's think it step-by-step

- 在analysis字段中，先对人物设定、人物特定以及语言风格进行分析
- 在'经历', '身份', '性格特点', '经典台词', '恋爱状态', '世界设定', '爱好', '不喜欢的事情',  '语言风格',  '口头禅', '技能和能力', '武器', '人物关系' 中选取最相关的5项，输出到most_related_items字段中
- 在claim字段中输出 "我将从dialogue中总结角色的a,b,c,d,e信息", a-e为most_related_items字段中的item
- 输出一个attributes字段，在这个字段下，将a,b,c,d,e的信息用json格式进行输出

"""

    return suffix_prompt

In [ ]:
attribute_detail = """### possible-items
- 经历，从对话中总结角色的经历
如"张东升是一个小城市少年宫的代课数学老师，因爱情来到妻子所在地工作。妻子执意离婚，张东升为逼迫妻子放弃离婚念头，将岳父母推下景区山崖，此行为被三个少年无意中拍摄到。"
- 身份
如"永安当的小伙计，后成为新安当的大老板，成为渝州巨贾"
- 恋爱状态
如"已婚，但妻子执意离婚", "对萧炎有特殊好感"
- 性格特点 :
如"智慧与信仰坚定，虽有时显得懦弱，但内心坚韧不拔；对弟子慈爱，尤其对孙悟空有时严厉，但出于对其成长的期望；对妖怪能施以慈悲，但也会坚持正义"
- 经典台词, 提取对话中能够显著反应人物个性的台词
如"['为了蜀山，我可以放弃一切。', '紫萱，我会守护你一生一世。', '剑出如龙，斩尽妖邪！']"
- 世界设定
如"故事在提瓦特大陆上展开，该世界分成七个国家，每个国家分别以一种元素为主题，并由对应元素的神明所分管。"
- 技能和能力
如"葵花点穴手、葵花解穴手、轻功（白三娘传授，天下第二）、隔空打穴、九九还阳掌（对佟湘玉使过）","血液具有特殊力量，能驱赶食人花","孤傲冷漠，有王者之霸气，好独来独往，对紫萱有柔情似水的一面，对景天有师父般的关怀"
- 爱好
如"古董，自由自在的生活，剑仙的传说"
"""

# 把角色打包成 list of str

In [ ]:
!pip install -q tiktoken datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("silk-road/50-Chinese-Novel-Characters")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:80: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
novel_list = []

novel2roles = {}

role2datas = {}

from tqdm import tqdm
for data in tqdm(dataset['train']):
    novel = data['book']
    role = data['role']
    if novel not in novel_list:
        novel_list.append(novel)

    if novel not in novel2roles:
        novel2roles[novel] = []

    if role not in novel2roles[novel]:
        novel2roles[novel].append(role)

    role_tuple = (novel, role)

    if role_tuple not in role2datas:
        role2datas[role_tuple] = []

    role2datas[role_tuple].append(data)

100%|██████████| 141985/141985 [00:26<00:00, 5294.85it/s]


In [ ]:
def conv2story( role, conversations ):
    for conv in conversations:
        conv["value"] = conv["value"].replace("\n", "  ")
    lines = [conv["value"] if conv["from"] == "human" else role + ": " + conv["value"] for conv in conversations]
    return "\n".join(lines)

In [ ]:
for novel, role in role2datas.keys():
    print(novel, role)
    datas = role2datas[(novel, role)]
    paired_story_index = [(conv2story(role, data["conversations"]), data["index"]) for data in datas]
    # Sort the pairs based on indexs (the second element of each pair)
    paired_story_index = sorted(paired_story_index, key=lambda x: x[1])

    sorted_stories = [pair[0] for pair in paired_story_index]
    break

超级兵王 郭璞


In [ ]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

def get_num_tokens(text):
    return len(enc.encode(text))

In [ ]:
import random

def sample_text_with_token( stoires , num_tokens = 3000 ):
    # token_per_story = [get_num_tokens(s) for s in stoires]
    n = len(stoires)

    process_indexes = []

    index = [i for i in range(n)]
    random.shuffle(index)

    current_tokens = 0

    ans = ""

    for i in index:
        current_story_token = get_num_tokens( stoires[i] )
        if current_tokens + current_story_token <= num_tokens:
            process_indexes.append(i)
            current_tokens += current_story_token
            ans += stoires[i] + "\n\n"
        else:
            break

    return ans.strip()







In [ ]:
save_datas = []

story_token = 4000

from tqdm import tqdm

narrators = ["叙述者", "旁白","文章作者","作者","Narrator","narrator"]

for novel, role in tqdm(role2datas.keys()):
    if role in narrators:
        continue
    # print(novel, role)
    datas = role2datas[(novel, role)]
    paired_story_index = [(conv2story(role, data["conversations"]), data["index"]) for data in datas]
    # Sort the pairs based on indexs (the second element of each pair)
    paired_story_index = sorted(paired_story_index, key=lambda x: x[1])

    sorted_stories = [pair[0] for pair in paired_story_index]

    n = len(sorted_stories)

    ranges = []

    if n < 10:
        ranges = [(0,n)]
    else:
        steps = n // 3
        ranges = [(i*steps, (i+1)*steps) for i in range(2)]
        ranges.append((2*steps, n))

    id_count = 0

    for left, right in ranges:
        whole_prompt = package_prefix_prompt(role, novel)

        whole_prompt += "\n" + attribute_detail + "\n"

        whole_prompt += "\n# dialogue\n" + sample_text_with_token(sorted_stories, story_token) + "\n"

        whole_prompt += package_suffix_prompt(role, novel)

        id_str = novel + "_" + role + "_" + str(id_count)

        save_data = {
            "id":id_str,
            "input":whole_prompt
        }

        save_datas.append(save_data)

        id_count = id_count + 1

100%|██████████| 510/510 [00:04<00:00, 114.63it/s]


In [ ]:
print(save_datas[12]["input"])

请帮助我为小说将夜中的 宁缺 角色总结他的人物设定

给定人物宁缺的对话数据，请为我总结宁缺的人物设定、人物特点以及语言风格，并输出成json格式
### possible-items
- 经历，从对话中总结角色的经历
如"张东升是一个小城市少年宫的代课数学老师，因爱情来到妻子所在地工作。妻子执意离婚，张东升为逼迫妻子放弃离婚念头，将岳父母推下景区山崖，此行为被三个少年无意中拍摄到。"
- 身份
如"永安当的小伙计，后成为新安当的大老板，成为渝州巨贾"
- 恋爱状态
如"已婚，但妻子执意离婚", "对萧炎有特殊好感"
- 性格特点 :
如"智慧与信仰坚定，虽有时显得懦弱，但内心坚韧不拔；对弟子慈爱，尤其对孙悟空有时严厉，但出于对其成长的期望；对妖怪能施以慈悲，但也会坚持正义"
- 经典台词, 提取对话中能够显著反应人物个性的台词
如"['为了蜀山，我可以放弃一切。', '紫萱，我会守护你一生一世。', '剑出如龙，斩尽妖邪！']"
- 世界设定
如"故事在提瓦特大陆上展开，该世界分成七个国家，每个国家分别以一种元素为主题，并由对应元素的神明所分管。"
- 技能和能力
如"葵花点穴手、葵花解穴手、轻功（白三娘传授，天下第二）、隔空打穴、九九还阳掌（对佟湘玉使过）","血液具有特殊力量，能驱赶食人花","孤傲冷漠，有王者之霸气，好独来独往，对紫萱有柔情似水的一面，对景天有师父般的关怀"
- 爱好
如"古董，自由自在的生活，剑仙的传说"


# dialogue
简大家: 我和你这孩子非亲非故，若不是心头一热，也懒得与你说这些话，所以你不要有什么抵触情绪，让你珍惜在书院里学习的机会，自不是害你。  *看着他严肃说道* 上次便与你说过，褚由贤这等富家公子可以玩，你一个穷酸少年却没有资格玩，今日更是如此，司徒小姐和金家小姐这些长安贵女可以玩，你还是没有资格玩。她们与你亲近，只是瞧着你好玩，对你暂时存着些好奇，这种意趣并不见得是恶意，但毕竟不是真的尊重。  如果你想成为她们真正的朋友，那么你就必须拥有一些值得她们尊重的能力与气度，如果你能走进书院二层楼，我相信世上所有的人都愿意做你的朋友。  *端起桌上那盏金线兰花露，轻啜一口润了润嗓子，然后抬起头来看着他继续平静说道* 以后来楼子里散心可以，但次数不要过频，酒更不能多喝。我本是风月行里一嬷嬷，自不会以为流连勾栏青楼是如何低贱的

In [ ]:
print(len(save_datas))

1344


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_name = "/content/drive/MyDrive/CardBuild/exp0212/card2extract.pkl"

In [ ]:
import pickle

with open(save_name, "wb") as f:
    pickle.dump(save_datas, f)